# TP PROJ 10 – SABR complet via CTMC et grilles non uniformes

Ce TP est dédié au dossier `PROJ/SABR/` de `PROJ_Option_Pricing_Matlab`, qui va **bien au‑delà** de la simple
approximation de volatilité implicite (Hagan) disponible dans `fypy/`.

Objectifs :
- comprendre la discrétisation CTMC du modèle SABR ;
- voir comment PROJ et les Q‑matrices sont utilisés pour des options européennes, américaines et barrières sous SABR ;
- faire le lien avec les approximations de vol implicite (que tu connais déjà côté Python).

## 1. Le dossier `PROJ/SABR`

Structure :
- `European_American_Barrier/` :
  - `SABR_EurBarAmer_func.m` : cœur du pricer SABR pour européennes / américaines / barrières ;
  - `Script_European_Barrier_American.m` : script principal de démonstration ;
- `Helper_Functions/` :
  - `General_Q_Matrix_Newest.m`, `getGenerator_Q_MatrixOnly.m` : construction de la Q‑matrix (générateur CTMC) pour le couple (log‑prix, vol) ;
  - `getNonUniformGrid.m` : génération de grilles non uniformes adaptées aux dynamiques SABR.

La philosophie :
- au lieu d’utiliser seulement une vol implicite approximée (Hagan‑Obloj), on discretise le **processus (F_t, σ_t)**,
  ou une transformation équivalente, sur une grille d’états ;
- on construit un générateur CTMC \( Q \) qui approxime la dynamique continue SABR ;
- on applique ensuite des techniques de type PROJ / exponentielle de matrices pour valoriser les options, y compris
  américaines et barrières.

## 2. Grilles non uniformes et Q‑Matrix

Le modèle SABR standard :
\[
\begin{aligned}
dF_t &= \alpha_t F_t^{\beta} dW_t^1, \\
d\alpha_t &= \nu \alpha_t dW_t^2, \\
dW_t^1 dW_t^2 &= \rho \, dt.
\end{aligned}
\]

Les difficultés :
- forte non‑linéarité en \( F_t^{\beta} \) ;
- volatilité \( \alpha_t \) strictement positive, dynamique multiplicative ;
- corrélation \( \rho \) non nulle entre prix et vol.

`getNonUniformGrid.m` :
- construit une grille d’états plus dense là où le processus passe le plus de temps (par ex. autour du forward, et
  pour des niveaux de vol réalistes) ;
- permet d’avoir une bonne précision avec un nombre d’états raisonnable.

`General_Q_Matrix_Newest.m`, `getGenerator_Q_MatrixOnly.m` :
- à partir des coefficients de diffusion SABR, construisent le générateur \( Q \) d’une chaîne de Markov continue qui
  approxime la dynamique en temps continu ;
- garantissent des propriétés de stabilité (positivité des probabilités, conservation de la masse).

## 3. Pricing d’options européennes, américaines et barrières sous SABR

`SABR_EurBarAmer_func.m` :
- prend en entrée les paramètres SABR (\( \alpha, \beta, \rho, \nu \)), la grille, la Q‑matrix et les caractéristiques
  du contrat (type d’option, barrière, maturité, etc.) ;
- résout les équations d’évaluation par :
  - exponentiation de la Q‑matrix entre dates d’exercice ;
  - programmation dynamique pour les Bermudans / américaines (max continuation vs exercice) ;
  - traitement spécifique des barrières (états absorbants, conditions aux limites).

`Script_European_Barrier_American.m` montre comment :
- choisir un jeu de paramètres SABR réaliste ;
- générer une grille non uniforme en (F, σ) ;
- construire la Q‑matrix ;
- pricer successivement :
  - des calls/puts européens ;
  - des options américaines ;
  - des options barrières (par exemple up‑and‑out) ;
- comparer avec l’approximation de Hagan (vol implicite) pour les européennes, afin de vérifier la cohérence.


## 4. Lien avec les approximations SABR dans `fypy`

Côté `fypy/`, SABR est utilisé essentiellement via une **approximation de volatilité implicite** (Hagan‑Obloj), puis
pluggué dans une formule Black–76. Ce TP met en avant l’approche **full SABR** :
- on respecte la dynamique complète du couple (F, σ) ;
- on peut pricer des produits path‑dépendants (barrières, américaines), ce qui est hors de portée d’une simple
  approximation de vol implicite ;
- le coût numérique est plus élevé, mais le cadre est beaucoup plus riche.

**Travail suggéré** :
- utiliser `Script_European_Barrier_American.m` pour calibrer un jeu de paramètres SABR à partir d’un smile synthétique ;
- comparer les prix européens obtenus via la Q‑matrix à ceux issus de Hagan‑Obloj ;
- étudier l’impact du raffinement de la grille non uniforme sur précision et temps de calcul.